In [2]:

import glob
from torchsummary import summary
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import json
from PIL import Image
from IPython.display import display
import random

test_text = '''
Do you have a problem that you are struggling to solve? Why don't you ask your friends for advice? When people ask for advice on solving a problem, often times they speak to more than one person. This is because different views are better for figuring out a tough problem, many opinions are better than one, and other people may have experienced a problem like yours and may be able to help you in making better decisions.
'''

C:\Users\Nutzer\AppData\Local\Temp\ipykernel_4456\2368509373.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# AI Text Image Recognition

Is there some similarity to Malware detection, where it's common to translate malware code to bytes and then images, which are then fed into NN/DL networks? Let's test a bit.

In [3]:
ai = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\4k\\generator_0_gpt2_4000_outputs.json", encoding='utf8'))
#ai = ai + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_1_daryl149_llama-2-7b-chat-hf_100_outputs.json", encoding='utf8'))
#ai = ai + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_1_gpt2_100_outputs.json", encoding='utf8'))

human = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\4k\\generator_1_gpt2_4000_outputs.json", encoding='utf8'))
#human = human + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_0_daryl149_llama-2-7b-chat-hf_100_outputs.json", encoding='utf8'))
#human = human + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_0_gpt2_100_outputs.json", encoding='utf8'))
print(len(ai))
print(len(human))

3988
3992


## Vis as Image


In [4]:
def output_to_image(detector_outputs):
    for output in detector_outputs:
        meta = output['metadata']
        height = meta['sample_rate']
        width = meta['sample_sequence_length']
        results = output['ensemble_results'][0]['sample_results']
        
        image = Image.new('RGBA', (100, 20), color=(0,0,0,255))
        pixels = image.load()

        x = 1
        offset_x = x
        y = 1

        for result in results:
            steps = result['model_outputs']['steps']
            steps.sort(key= lambda x: x['step'])

            for step in steps:                
                r = 255
                g = 255
                b = 255
                
                all_probs = step['top_k_probs']
                all_probs.sort(reverse=True)
                other_probs = all_probs[1:]
                
                # mid
                pixels[x, y] = (r, g, b, int(step['target_prob'] * 255))
                # right
                pixels[x+1, y] = (r, g, b, int(other_probs[0] * 255))
                # left
                pixels[x-1, y] = (r, g, b, int(other_probs[1] * 255))
                # down
                pixels[x, y-1] = (r, g, b, int(other_probs[2] * 255))
                # up
                pixels[x, y+1] = (r, g, b, int(other_probs[3] * 255))
                
                x += 3
            
            offset_x = int((offset_x % 2 == 1))
            x = offset_x
            y += 3
        display(image)

def draw_image(width, height):
    image = Image.new('RGBA', (width, height))
    pixels = image.load()
    # iterate over all pixels
    for y in range(height):
        for x in range(width):
            r = random.randint(0, 255)
            g = random.randint(0, 255)
            b = random.randint(0, 255)
            a = random.randint(0, 1)
            pixels[x, y] = (r, g, b, int(a * 255))  # Set the pixel RGBA values
    return image
    

In [5]:
output_to_image(human[:2])
output_to_image(ai[:2])

# CNN

Let's try some CNN detection on the steps

## 2D CNN

Let's start by creating 2d arrays with x channels per "pixel"

In [21]:
def create_dataset(detector_outputs, label):
    data = []
    labels = []
    
    for output in detector_outputs:
        results = output['ensemble_results'][0]['sample_results']
        rows = []

        for result in results:
            steps = result['model_outputs']['steps']
            steps.sort(key= lambda x: x['step'])
            columns = []
            for step in steps:
                channels = [step['target_prob']]
                top_k_probs = step['top_k_probs']
                top_k_probs.sort()
                channels = channels + top_k_probs
                columns.append(channels)
                
            rows.append(columns)
          
        data.append(rows)
        labels.append(label)  
    data = np.array(data)
    labels = np.array(labels)
    
    return data, labels

In [22]:
data_ai, labels_ai = create_dataset(ai, 1)
data_hu, labels_hu = create_dataset(human, 0)
data = np.concatenate((data_ai, data_hu), axis=0)
labels = np.concatenate((labels_hu, labels_ai))
print(len(labels))

# Convert numpy arrays to tensors
# We need: [batch_size, channels, height, width]
# data_tensor = torch.tensor(data, dtype=torch.float32).permute(0, 2, 1).unsqueeze(-2)
data_tensor = torch.tensor(data, dtype=torch.float32).permute(0, 3, 1, 2)
print(data_tensor.shape)
labels_tensor = torch.tensor(labels, dtype=torch.long)

print(data_tensor[:1])
print(labels_tensor[:1])

7980
torch.Size([7980, 11, 6, 32])
tensor([[[[5.5349e-02, 3.1169e-02, 6.8112e-02,  ..., 3.6298e-02,
           2.9099e-02, 9.1484e-01],
          [9.0897e-04, 5.1601e-01, 1.0902e-02,  ..., 1.0621e-02,
           8.6073e-02, 5.9349e-03],
          [4.2450e-02, 8.3047e-01, 6.8794e-02,  ..., 4.2377e-03,
           1.8796e-02, 2.9923e-02],
          [2.5172e-04, 1.7383e-02, 4.3808e-04,  ..., 9.9983e-01,
           1.4946e-01, 4.2450e-02],
          [2.2887e-01, 1.5550e-01, 4.9840e-02,  ..., 6.5941e-02,
           2.3018e-02, 1.3634e-01],
          [1.3649e-01, 3.6298e-02, 2.9099e-02,  ..., 5.3489e-01,
           2.2832e-02, 6.4482e-03]],

         [[2.0131e-04, 2.0213e-04, 4.0885e-06,  ..., 2.3299e-06,
           1.5963e-05, 7.3132e-05],
          [6.2795e-06, 4.0680e-04, 8.4170e-04,  ..., 2.6957e-04,
           1.8569e-04, 2.4884e-05],
          [2.6726e-04, 1.2189e-04, 5.4611e-04,  ..., 8.9511e-05,
           4.2483e-04, 6.4260e-05],
          [5.0063e-04, 1.8412e-05, 1.8213e-05,  ..., 2

# Train, Test Split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2, random_state=42)
print("x_train shape",X_train.shape)
print("x_test shape",X_test.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)
print(y_test[:20])

x_train shape torch.Size([6384, 11, 6, 32])
x_test shape torch.Size([1596, 11, 6, 32])
y_train shape torch.Size([6384])
y_test shape torch.Size([1596])
tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0])


# Model


In [60]:
model = nn.Sequential(
    nn.Conv2d(in_channels=11, out_channels=32, kernel_size=(2,8), padding=(0, 1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(1,2), stride=(1, 2)),
    nn.Dropout(p=0.25),

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2,6), padding=(0,1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(1,2), stride=(1,2)),
    nn.Dropout(p=0.25),

    nn.Flatten(),
    nn.Linear(in_features=20, out_features=64),
    nn.ReLU(),
    nn.Dropout(p=0.4),

    nn.Linear(in_features=64, out_features=1),
    nn.Sigmoid()
)

print(model)

Sequential(
  (0): Conv2d(11, 32, kernel_size=(2, 8), stride=(1, 1), padding=(0, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (3): Dropout(p=0.25, inplace=False)
  (4): Conv2d(32, 64, kernel_size=(2, 6), stride=(1, 1), padding=(0, 1))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (7): Dropout(p=0.25, inplace=False)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=20, out_features=64, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.4, inplace=False)
  (12): Linear(in_features=64, out_features=1, bias=True)
  (13): Sigmoid()
)


In [61]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [62]:
# Training the model
num_epochs = 150
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    model.train()  # Set the model to training mode
    for inputs, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        
        if len(labels.size()) == 0:
            labels = labels.unsqueeze(0).expand(64, 1).float()
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Calculate training accuracy
        predicted = outputs > 0.5  # Assuming threshold of 0.5 for binary classification
        total += labels.size(0)
        correct += (predicted == labels.byte()).sum().item()
    
    # Print training loss and accuracy
    train_loss = running_loss / len(y_train)
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}, Training Loss: {train_loss:0.5f}, Training Accuracy: {train_accuracy:0.5f}")
    
    # Test loss and accuracy calculation
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for test_inputs, test_labels in zip(X_test, y_test):
            test_outputs = model(test_inputs)
            if len(test_labels.size()) == 0:
                test_labels = test_labels.unsqueeze(0).expand(64, 1).float()
            test_loss += criterion(test_outputs, test_labels).item()
            test_predicted = test_outputs > 0.5
            test_total += test_labels.size(0)
            test_correct += (test_predicted == test_labels.byte()).sum().item()
    
    # Print test loss and accuracy
    test_loss /= len(y_test)
    test_accuracy = test_correct / test_total
    print(f"Epoch {epoch+1}, Test Loss: {test_loss:0.5f}, Test Accuracy: {test_accuracy:0.5f}\n")


Epoch 1, Training Loss: 0.54221, Training Accuracy: 0.72952
Epoch 1, Test Loss: 0.44469, Test Accuracy: 0.79721

Epoch 2, Training Loss: 0.44992, Training Accuracy: 0.80426
Epoch 2, Test Loss: 0.39902, Test Accuracy: 0.82649

Epoch 3, Training Loss: 0.43061, Training Accuracy: 0.81784
Epoch 3, Test Loss: 0.39756, Test Accuracy: 0.82655

Epoch 4, Training Loss: 0.42200, Training Accuracy: 0.82142
Epoch 4, Test Loss: 0.39339, Test Accuracy: 0.83119

Epoch 5, Training Loss: 0.40873, Training Accuracy: 0.83153
Epoch 5, Test Loss: 0.36601, Test Accuracy: 0.84655

Epoch 6, Training Loss: 0.39243, Training Accuracy: 0.83937
Epoch 6, Test Loss: 0.36588, Test Accuracy: 0.84415

Epoch 7, Training Loss: 0.38756, Training Accuracy: 0.84238
Epoch 7, Test Loss: 0.36241, Test Accuracy: 0.84544

Epoch 8, Training Loss: 0.36124, Training Accuracy: 0.85716
Epoch 8, Test Loss: 0.32694, Test Accuracy: 0.86352

Epoch 9, Training Loss: 0.34729, Training Accuracy: 0.86362
Epoch 9, Test Loss: 0.32182, Test Ac